# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [2]:
import pandas as pd
import requests

# Carregando a base de dados
sinasc = 'SINASC_RO_2019.csv'
df = pd.read_csv(sinasc)

# Contando o número total de registros
num_registros_total = len(df)

# Contando o número de registros não duplicados
num_registros_nao_duplicados = len(df.drop_duplicates())

# Verificando se há linhas duplicadas
linhas_duplicadas = df[df.duplicated()]

# Exibindo os resultados
print(f"Número total de registros: {num_registros_total}")
print(f"Número de registros não duplicados: {num_registros_nao_duplicados}")

if not linhas_duplicadas.empty:
    print("\nLinhas duplicadas encontradas:")
    print(linhas_duplicadas)
else:
    print("\nNão há linhas duplicadas na base de dados.")

Número total de registros: 27028
Número de registros não duplicados: 27028

Não há linhas duplicadas na base de dados.


In [3]:
# Contando o número de valores missing por variável
valores_missing_por_variavel = df.isnull().sum()

# Exibindo os resultados
print("Número de valores missing por variável:")
print(valores_missing_por_variavel)


Número de valores missing por variável:
ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64


In [4]:
# Lista de colunas desejadas
colunas_desejadas = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

# Criando um novo DataFrame apenas com as colunas desejadas
df_selecionado = df[colunas_desejadas]

# Exibindo as primeiras linhas do novo DataFrame
print(df_selecionado.head())


   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1        1        29        2.0      8 a 11 anos         1.0  37 a 41 semanas   
2        1        37        9.0      8 a 11 anos         2.0  37 a 41 semanas   
3        1        30        5.0  12 anos ou mais         0.0  37 a 41 semanas   
4        1        30        2.0      8 a 11 anos         1.0  37 a 41 semanas   

  GRAVIDEZ  CONSULTAS  APGAR5  
0    Única          4    10.0  
1    Única          4     9.0  
2    Única          4    10.0  
3    Única          3    10.0  
4    Única          4    10.0  


In [5]:
# Contando o número de valores missing por variável no novo DataFrame
valores_missing_por_variavel = df_selecionado.isnull().sum()

# Exibindo os resultados
print("Número de valores missing por variável no DataFrame selecionado:")
print(valores_missing_por_variavel)


Número de valores missing por variável no DataFrame selecionado:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [6]:
# Removendo registros com valores ausentes em 'APGAR5'
df_sem_apgar5_missing = df_selecionado.dropna(subset=['APGAR5'])

# Contando o número total de registros após a remoção
num_registros_sem_apgar5_missing = len(df_sem_apgar5_missing)

# Contando o número de valores missing por variável no novo DataFrame
valores_missing_por_variavel_sem_apgar5 = df_sem_apgar5_missing.isnull().sum()

# Exibindo os resultados
print("Número total de registros após a remoção de APGAR5 missing:", num_registros_sem_apgar5_missing)
print("\nNúmero de valores missing por variável no DataFrame sem APGAR5 missing:")
print(valores_missing_por_variavel_sem_apgar5)


Número total de registros após a remoção de APGAR5 missing: 26925

Número de valores missing por variável no DataFrame sem APGAR5 missing:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [10]:
# Substituindo valores faltantes em 'QTDFILVIVO' por zero
df_selecionado['QTDFILVIVO'].fillna(0, inplace=True)

# Substituindo valores faltantes e valores iguais a 9 por zero em 'ESTCIVMAE' e 'CONSULTAS'
df_selecionado[['ESTCIVMAE', 'CONSULTAS']] = df_selecionado[['ESTCIVMAE', 'CONSULTAS']].replace({9: 0, pd.NA: 0})

# Exibindo as primeiras linhas do DataFrame resultante
print(df_selecionado.head())


   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO         GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0  37 a 41 semanas   
1        1        29        2.0      8 a 11 anos         1.0  37 a 41 semanas   
2        1        37        0.0      8 a 11 anos         2.0  37 a 41 semanas   
3        1        30        5.0  12 anos ou mais         0.0  37 a 41 semanas   
4        1        30        2.0      8 a 11 anos         1.0  37 a 41 semanas   

  GRAVIDEZ  CONSULTAS  APGAR5  
0    Única          4    10.0  
1    Única          4     9.0  
2    Única          4    10.0  
3    Única          3    10.0  
4    Única          4    10.0  


C:\Users\mardo\AppData\Local\Temp\ipykernel_20852\2133985999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado['QTDFILVIVO'].fillna(0, inplace=True)
C:\Users\mardo\AppData\Local\Temp\ipykernel_20852\2133985999.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado[['ESTCIVMAE', 'CONSULTAS']] = df_selecionado[['ESTCIVMAE', 'CONSULTAS']].replace({9: 0, pd.NA: 0})


In [11]:

#'LOCNASC': Como 'LOCNASC' parece ser uma variável categórica que representa o local de nascimento, substituir os valores faltantes por um valor 'mais provável' pode ser apropriado. Isso pode ser feito usando o valor mais comum ou a moda da variável.

#'IDADEMAE': Para 'IDADEMAE', que representa a idade da mãe, substituir os valores faltantes por um valor 'mais provável' pode ser arriscado, pois a idade da mãe pode variar significativamente. Nesse caso, substituir por um valor como a média ou a mediana das idades existentes pode ser uma escolha mais razoável.

#'ESCMAE': Se 'ESCMAE' representa a escolaridade da mãe, você pode substituir os valores faltantes por um valor 'mais provável', como a moda, se essa informação for relevante e puder ser estimada com segurança.

#'GESTACAO': Se 'GESTACAO' representa a duração da gestação, a substituição por um valor 'mais provável', como a média ou mediana, pode ser razoável, desde que a distribuição dos dados não seja altamente distorcida.

#'GRAVIDEZ': Para 'GRAVIDEZ', que representa a quantidade de gravidezes, você pode substituir os valores faltantes por zero, assumindo que a ausência de informação indica nenhuma gravidez anterior. Essa é uma suposição comum nesses casos, mas a escolha depende do conhecimento específico do domínio.


In [14]:
# Como 'LOCNASC' é uma variável categórica que representa o local de nascimento, vou substituir os valores faltantes por um valor 'mais provável'. 
# Preenchendo valores faltantes em 'LOCNASC' com a moda (valor mais comum)
df_selecionado['LOCNASC'].fillna(df_selecionado['LOCNASC'].mode()[0], inplace=True)

# Para 'IDADEMAE', que representa a idade da mãe, substituir os valores faltantes por um valor 'mais provável' é arriscado, pois a idade da mãe pode variar significativamente. Nesse caso, vou substituir por um valor como a média ou a mediana das idades existentes
# Preenchendo valores faltantes em 'IDADEMAE' com a mediana
df_selecionado['IDADEMAE'].fillna(df_selecionado['IDADEMAE'].median(), inplace=True)

# 'ESCMAE'representa a escolaridade da mãe, vou substituir os valores faltantes por um valor 'mais provável'
# Preenchendo valores faltantes em 'ESCMAE' com a moda
df_selecionado['ESCMAE'].fillna(df_selecionado['ESCMAE'].mode()[0], inplace=True)

# 'GESTACAO' representa a duração da gestação, a substituição por um valor 'mais provável', como a média ou mediana, pode ser razoável
# Convertendo a coluna 'GESTACAO' para numérica e preenchendo valores faltantes com a média
df_selecionado['GESTACAO'] = pd.to_numeric(df_selecionado['GESTACAO'], errors='coerce')
df_selecionado['GESTACAO'].fillna(df_selecionado['GESTACAO'].mean(), inplace=True)

# 'GRAVIDEZ' representa a quantidade de gravidezes, voue substituir os valores faltantes por zero, assumindo que a ausência de informação indica nenhuma gravidez anterior. 
# Preenchendo valores faltantes em 'GRAVIDEZ' com zero
df_selecionado['GRAVIDEZ'].fillna(0, inplace=True)

# Exibindo as primeiras linhas do DataFrame resultante
print(df_selecionado.head())


   LOCNASC  IDADEMAE  ESTCIVMAE           ESCMAE  QTDFILVIVO  GESTACAO  \
0        1        19        5.0      8 a 11 anos         0.0       NaN   
1        1        29        2.0      8 a 11 anos         1.0       NaN   
2        1        37        0.0      8 a 11 anos         2.0       NaN   
3        1        30        5.0  12 anos ou mais         0.0       NaN   
4        1        30        2.0      8 a 11 anos         1.0       NaN   

  GRAVIDEZ  CONSULTAS  APGAR5  
0    Única          4    10.0  
1    Única          4     9.0  
2    Única          4    10.0  
3    Única          3    10.0  
4    Única          4    10.0  


C:\Users\mardo\AppData\Local\Temp\ipykernel_20852\2507646485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado['LOCNASC'].fillna(df_selecionado['LOCNASC'].mode()[0], inplace=True)
C:\Users\mardo\AppData\Local\Temp\ipykernel_20852\2507646485.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selecionado['IDADEMAE'].fillna(df_selecionado['IDADEMAE'].median(), inplace=True)
C:\Users\mardo\AppData\Local\Temp\ipykernel_20852\2507646485.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [15]:
# Criando a coluna de categorização do Apgar
bins = [-1, 3, 5, 7, 10]
labels = ['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal']
df['APGAR_categoria'] = pd.cut(df['APGAR5'], bins=bins, labels=labels, right=False)

# Calculando as frequências da categorização do Apgar
frequencias_apgar_categoria = df['APGAR_categoria'].value_counts()

# Exibindo as frequências
print("Frequências da categorização do Apgar:")
print(frequencias_apgar_categoria)


Frequências da categorização do Apgar:
normal              18311
asfixia leve          124
asfixia severa         59
asfixia moderada       38
Name: APGAR_categoria, dtype: int64


In [16]:
# Renomeando as variáveis para snake case
df.rename(columns={
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5'
}, inplace=True)

# Exibindo as primeiras linhas do DataFrame resultante
print(df.head())

   ORIGEM   CODESTAB  CODMUNNASC  loc_nasc  idade_mae  est_civ_mae  \
0       1  2679477.0      110001         1         19          5.0   
1       1  2679477.0      110001         1         29          2.0   
2       1  2679477.0      110001         1         37          9.0   
3       1  2516500.0      110001         1         30          5.0   
4       1  2516500.0      110001         1         30          2.0   

           esc_mae  CODOCUPMAE  qtd_fil_vivo  QTDFILMORT  ...  CONTADOR  \
0      8 a 11 anos         NaN           0.0         0.0  ...         1   
1      8 a 11 anos    999992.0           1.0         0.0  ...         2   
2      8 a 11 anos    513205.0           2.0         0.0  ...         3   
3  12 anos ou mais    231205.0           0.0         0.0  ...         4   
4      8 a 11 anos    999992.0           1.0         0.0  ...         5   

  munResStatus munResTipo               munResNome  munResUf munResLat  \
0        ATIVO      MUNIC    Alta Floresta D'Oeste  Ro